# Setup

Environment setup for PPO-LLM experiments. Run this notebook first.

In [ ]:
%cd /content
!rm -rf overcooked_ai
!git clone --depth 1 https://github.com/HumanCompatibleAI/overcooked_ai.git /content/overcooked_ai > /dev/null

import sys, os
sys.path.append("/content/overcooked_ai/src")

In [ ]:
!pip install -q gym "stable-baselines3[extra]" overcooked-ai transformers

In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4

import os
os._exit(0)

In [ ]:
!pip install -q "shimmy>=2.0"

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

import os, zipfile

DRIVE_ROOT = "/content/drive/MyDrive"
RUNS_ROOT = os.path.join(DRIVE_ROOT, "runs")

if (not os.path.exists(RUNS_ROOT)) and os.path.exists(os.path.join(DRIVE_ROOT, "runs.zip")):
    with zipfile.ZipFile(os.path.join(DRIVE_ROOT, "runs.zip"), "r") as zf:
        zf.extractall(DRIVE_ROOT)

## Configuration

In [ ]:
import torch
from overcooked_ai_py.mdp.actions import Action

LAYOUT = "asymmetric_advantages"
HORIZON = 400
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SEEDS = [1001, 2002, 3003, 4004, 5005]

CHECKPOINT_EVERY_STEPS = 50_000
LOG_EVERY_STEPS = 2_048
EVAL_EPISODES = 10

RESULTS_CSV = "/content/drive/MyDrive/results_combined_new.csv"
RUNS_DIR = "/content/drive/MyDrive/runs"

print(f"Device: {DEVICE}")